In [1]:
# Reda Alhamza, 201365390                                   Ch7 Assignment                                                       211-MIS-545-02

In [2]:
import pandas as pd

In [3]:
Loans_Training = pd.read_csv("D:/DatasetsP/Loans_Training")

In [4]:
Loans_Test = pd.read_csv("D:/DatasetsP/Loans_Test")

# Q1: Using the training data set, create a C5.0 model (call it Model 1) to predict a loan applicant's Approval using Debt‐to‐Income Ratio, FICO Score, and Request Amount. Obtain the predicted responses. 

In [5]:
Loans_Training['Approval'].replace({'T':1, 'F':0}, inplace=True)
Loans_Test['Approval'].replace({'T':1, 'F':0}, inplace=True)

In [6]:
y = Loans_Training[['Approval']] # outcome variable

In [7]:
X = Loans_Training[['Debt-to-Income Ratio', 'FICO Score', 'Request Amount']] # Predictors

In [8]:
import statsmodels.tools.tools as stattools 
from sklearn.tree import DecisionTreeClassifier , export_graphviz # To be able to use DecisionTreeClassifier, import this package.

In [9]:
Model_1 = DecisionTreeClassifier(criterion="entropy", min_samples_split =75,max_leaf_nodes =5).fit(X,y)

In [10]:
Model_1_predict = Model_1.predict(Loans_Test[['Debt-to-Income Ratio', 'FICO Score', 'Request Amount']])
Model_1_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# Q2: Evaluate Model 1 using the test data set. Construct a contingency table to compare the actual and predicted values of Approval. 

In [11]:
con_table = pd.crosstab(Loans_Test['Approval'], Model_1_predict)
con_table

col_0,0,1
Approval,,
0,17665,7269
1,751,24013


# Q3: Do the following to construct the simplified data‐driven cost matrix: 
# a. Compute the mean of the Interest per loan applicant from the training data set. Set the negative of that value to be the “cost” of a true positive. Calculate the mean Request Amount per loan applicant from the training data set. Set this value to be the cost of a false positive.

In [12]:
TP_C = -round(Loans_Training['Interest'].mean(),2)
FP_C = round(Loans_Training['Request Amount'].mean(),2)
TN_C = 0
FN_C = 0

In [13]:
A = pd.DataFrame([[TN_C, FP_C],[FN_C, TP_C]])
A

,0,1
0,0,13427.08
1,0,-6042.19


# b. Obtain the simplified data‐driven cost matrix.

In [14]:
cost_r = pd.DataFrame([[A.iloc[0,0]/A.iloc[0,1], A.iloc[0,1]/A.iloc[0,1]],[A.iloc[1,0]/A.iloc[0,1]-A.iloc[1,1]/A.iloc[0,1], A.iloc[1,1]/A.iloc[0,1]-A.iloc[1,1]/A.iloc[0,1]]])
cost_r  

,0,1
0,0.00,1.0
1,0.45,0.0


# Q4: For Model 1, calculate all of the model evaluation measures as shown in the table below. Call this table the Model Evaluation Table.

In [15]:
def model_eval(matrix, matrix1, model_name ):
    tn = matrix.iloc[0,0]
    tp = matrix.iloc[1,1]
    fn = matrix.iloc[1,0]
    fp = matrix.iloc[0,1]
    tap = fn+tp
    tan = tn+fp
    tpn = tn+fn
    tpp = fp+tp
    precision = tp / tpp
    recall = tp / tap
    total = tn+tp+fn+fp
    tn_c = matrix1.iloc[0,0]
    fn_c = matrix1.iloc[1,0]
    fp_c = matrix1.iloc[0,1]  
    tp_c = matrix1.iloc[1,1] 
    
    
    data = [
        round((tp+tn)/total,4),
        round(1-((tp+tn)/total),4),
        round(tp /tap,4),
        round(tn /tan,4),
        round(precision,4),
        round(2 * (precision * recall) / (precision + recall),4),
        round(5 * (precision * recall) / ((4 * precision) + recall),4),
        round(1.25 * (precision * recall) / ((.25 * precision) + recall),4),
        round((fn*fn_c + tn*tn_c + fp*fp_c + tp*tp_c),4), # Overall Model Cost
        round(-(fn*fn_c + tn*tn_c + fp*fp_c + tp*tp_c)/total,4), # Model Profit per Record
                                                                                ]
    return(
        pd.DataFrame(data, columns=[model_name],
        index = ['Accuracy','ErrorRate','Sensitivity','Specificity','Precision','F1','F2','F3','Overall Cost','Profit per Customer'])
                                                                                                 )

In [16]:
Model= model_eval(con_table, A , model_name = 'Model 1')
Model

,Model 1
Accuracy,8.386000e-01
ErrorRate,1.614000e-01
Sensitivity,9.697000e-01
Specificity,7.085000e-01
Precision,7.676000e-01
F1,8.569000e-01
F2,9.212000e-01
F3,8.010000e-01
Overall Cost,-4.748966e+07
Profit per Customer,9.555649e+02


# Q5: Using the training set, build a C5.0 model (call it Model 2) to predict a loan applicant's Approval using Debt‐to‐Income Ratio, FICO Score, and Request Amount, using the simplified data‐driven cost matrix. Populate the Model Evaluation Table with the evaluation measures for Model 2, using the data‐driven cost matrix as shown in the results below. 

In [17]:
cost_matrix = {0:A.iloc[0,1]/A.iloc[0,1], 1:A.iloc[1,0]/A.iloc[0,1]-A.iloc[1,1]/A.iloc[0,1]}
Model2 = DecisionTreeClassifier(criterion="entropy", min_samples_split =75, max_leaf_nodes =5, class_weight =cost_matrix).fit(X,y)

Model2_predict = Model2.predict(Loans_Test[['Debt-to-Income Ratio', 'FICO Score', 'Request Amount']])

con_table2 = pd.crosstab(Loans_Test['Approval'], Model2_predict)

In [18]:
Model_2 = model_eval(con_table2, A , model_name = 'Model 2')
Model_2

,Model 2
Accuracy,8.210000e-01
ErrorRate,1.790000e-01
Sensitivity,8.096000e-01
Specificity,8.324000e-01
Precision,8.275000e-01
F1,8.185000e-01
F2,8.131000e-01
F3,8.239000e-01
Overall Cost,-6.502072e+07
Profit per Customer,1.308316e+03


In [19]:
All_Models = pd.concat([Model, Model_2], axis=1)
All_Models

,Model 1,Model 2
Accuracy,8.386000e-01,8.210000e-01
ErrorRate,1.614000e-01,1.790000e-01
Sensitivity,9.697000e-01,8.096000e-01
Specificity,7.085000e-01,8.324000e-01
Precision,7.676000e-01,8.275000e-01
F1,8.569000e-01,8.185000e-01
F2,9.212000e-01,8.131000e-01
F3,8.010000e-01,8.239000e-01
Overall Cost,-4.748966e+07,-6.502072e+07
Profit per Customer,9.555649e+02,1.308316e+03


# For Profit per Customer, I belive there is a mistake if you can see page 106 in the textbook. Where the profit per Record = - Model Cost per Record 